In [57]:
# Data Wrangling
import os
import numpy as np
import pandas as pd
import shutil
from collections import Counter
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from PIL import Image
import plotly.graph_objects as go

# 
import tensorflow as tf
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Dense, Dropout, BatchNormalization, Input

import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import plotly.figure_factory as ff


In [45]:
folders_to_clear = ['./data/simpsons_split', './data/simpsons_top_18']

for folder in folders_to_clear:
    if os.path.exists(folder):
        shutil.rmtree(folder)
        print(f"Eliminado: {folder}")
    else:
        print(f"No existe: {folder}")


Eliminado: ./data/simpsons_split
Eliminado: ./data/simpsons_top_18


In [46]:
DATA_PATH = './data/simpsons'

# read filenames for images in the data folder
filenames = [f for f in os.listdir(DATA_PATH) if f.endswith('.jpg')]

# extract character names before _pic
classes = [filename.split('_pic')[0] for filename in filenames]

# Count class distribution
class_counts = Counter(classes)

# print the distribution
for cl, count in class_counts.most_common():
    print(f"{cl}: {count}")

homer_simpson: 2246
ned_flanders: 1454
moe_szyslak: 1452
lisa_simpson: 1354
bart_simpson: 1342
marge_simpson: 1291
krusty_the_clown: 1206
principal_skinner: 1194
charles_montgomery_burns: 1193
milhouse_van_houten: 1079
chief_wiggum: 986
abraham_grampa_simpson: 913
sideshow_bob: 877
apu_nahasapeemapetilon: 623
kent_brockman: 498
comic_book_guy: 469
edna_krabappel: 457
nelson_muntz: 358
lenny_leonard: 310
mayor_quimby: 246
waylon_smithers: 181
maggie_simpson: 128
groundskeeper_willie: 121
barney_gumble: 106
selma_bouvier: 103
carl_carlson: 98
ralph_wiggum: 89
patty_bouvier: 72
martin_prince: 71
professor_john_frink: 65
snake_jailbird: 55
cletus_spuckler: 47
rainier_wolfcastle: 45
agnes_skinner: 42
sideshow_mel: 40
otto_mann: 32
fat_tony: 27
gil: 27
miss_hoover: 17
disco_stu: 8
troy_mcclure: 8
lionel_hutz: 3


In [47]:
# only get the top 18 classes
top_classes = set([cl for cl, _ in class_counts.most_common(18)])

# make a folder with only selected images
FILTERED_PATH = 'data/simpsons_top_18'
os.makedirs(FILTERED_PATH, exist_ok=True)

# Move top_18 images into a folder simpsons_filtered/{class}/image
for filename in filenames:
    cl = filename.split('_pic')[0]
    if cl in top_classes:
        src = os.path.join(DATA_PATH, filename)
        target_dir = os.path.join(FILTERED_PATH, cl)
        os.makedirs(target_dir, exist_ok=True)
        shutil.copy(src, os.path.join(target_dir, filename))

In [48]:
SPLIT_PATH = './data/simpsons_split'
os.makedirs(SPLIT_PATH, exist_ok=True)

for cls in os.listdir(FILTERED_PATH):
    class_dir = os.path.join(FILTERED_PATH, cls)  
    if not os.path.isdir(class_dir):
        continue

    images = [f for f in os.listdir(class_dir) if f.endswith('.jpg')]
    trainval_imgs, test_imgs = train_test_split(images, test_size=0.1, random_state=333)
    train_imgs, val_imgs = train_test_split(trainval_imgs, test_size=0.2, random_state=333)

    for subset, subset_imgs in zip(['train', 'val', 'test'], [train_imgs, val_imgs, test_imgs]):
        subset_dir = os.path.join(SPLIT_PATH, subset, cls)
        os.makedirs(subset_dir, exist_ok=True)
        for img in subset_imgs:
            src_path = os.path.join(class_dir, img)  
            dst_path = os.path.join(subset_dir, img)
            shutil.copy2(src_path, dst_path)

In [49]:
for i in range(5):
    with Image.open(os.path.join(FILTERED_PATH, 'homer_simpson', f'homer_simpson_pic_0000.jpg')) as img:
        print('Image sizes examples: ', img.size)

Image sizes examples:  (288, 416)
Image sizes examples:  (288, 416)
Image sizes examples:  (288, 416)
Image sizes examples:  (288, 416)
Image sizes examples:  (288, 416)


In [50]:
IMG_SIZE = (128, 192)
BATCH_SIZE = 32

# only augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# only normalization por validation and test
val_test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    './data/simpsons_split/train',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

val_gen = val_test_datagen.flow_from_directory(
    './data/simpsons_split/val',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

test_gen = val_test_datagen.flow_from_directory(
    './data/simpsons_split/test',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)


Found 13658 images belonging to 18 classes.
Found 3426 images belonging to 18 classes.
Found 1908 images belonging to 18 classes.


In [51]:
train_gen.class_indices

{'abraham_grampa_simpson': 0,
 'apu_nahasapeemapetilon': 1,
 'bart_simpson': 2,
 'charles_montgomery_burns': 3,
 'chief_wiggum': 4,
 'comic_book_guy': 5,
 'edna_krabappel': 6,
 'homer_simpson': 7,
 'kent_brockman': 8,
 'krusty_the_clown': 9,
 'lisa_simpson': 10,
 'marge_simpson': 11,
 'milhouse_van_houten': 12,
 'moe_szyslak': 13,
 'ned_flanders': 14,
 'nelson_muntz': 15,
 'principal_skinner': 16,
 'sideshow_bob': 17}

In [52]:
train_gen.image_shape

(128, 192, 3)

In [58]:
num_classes = train_gen.num_classes 
INPUT_SHAPE = (IMG_SIZE[0], IMG_SIZE[1], 3)

model = Sequential([
    Input(shape=(128, 192, 3)),

    # Bloque 1
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(),
    Dropout(0.25),

    # Bloque 2
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(),
    Dropout(0.25),

    # Bloque 3
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(),
    Dropout(0.25),

    # Clasificación
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(train_gen.num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        filepath='best_model.keras',
        save_best_only=True,
        monitor='val_loss',
        verbose=1
    )
]


model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_39 (Conv2D)              │ (None, 128, 192, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_27          │ (None, 128, 192, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_40 (Conv2D)              │ (None, 128, 192, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_27 (MaxPooling2D) │ (None, 64, 96, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 64, 96, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_41 (Conv2D)              │ (None, 64, 96, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_28          │ (None, 64, 96, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_42 (Conv2D)              │ (None, 64, 96, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_28 (MaxPooling2D) │ (None, 32, 48, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 32, 48, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_43 (Conv2D)              │ (None, 32, 48, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_29          │ (None, 32, 48, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_44 (Conv2D)              │ (None, 32, 48, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_29 (MaxPooling2D) │ (None, 16, 24, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 16, 24, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 18)             │         4,626 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 325,554 (1.24 MB)

 Trainable params: 325,106 (1.24 MB)

 Non-trainable params: 448 (1.75 KB)

In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    callbacks=callbacks
)

/Users/marco.vgn/.pyenv/versions/UNAM_diplomado/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
427/427 ━━━━━━━━━━━━━━━━━━━━ 0s 983ms/step - accuracy: 0.1463 - loss: 2.7334
Epoch 1: val_loss improved from inf to 2.22546, saving model to best_model.keras
427/427 ━━━━━━━━━━━━━━━━━━━━ 446s 1s/step - accuracy: 0.1464 - loss: 2.7329 - val_accuracy: 0.3079 - val_loss: 2.2255
Epoch 2/20
427/427 ━━━━━━━━━━━━━━━━━━━━ 0s 979ms/step - accuracy: 0.4171 - loss: 1.8949
Epoch 2: val_loss improved from 2.22546 to 1.61116, saving model to best_model.keras
427/427 ━━━━━━━━━━━━━━━━━━━━ 443s 1s/step - accuracy: 0.4173 - loss: 1.8944 - val_accuracy: 0.5414 - val_loss: 1.6112
Epoch 3/20
427/427 ━━━━━━━━━━━━━━━━━━━━ 0s 982ms/step - accuracy: 0.6325 - loss: 1.2352
Epoch 3: val_loss improved from 1.61116 to 1.00777, saving model to best_model.keras
427/427 ━━━━━━━━━━━━━━━━━━━━ 444s 1s/step - accuracy: 0.6326 - loss: 1.2350 - val_accuracy: 0.7093 - val_loss: 1.0078
Epoch 4/20
427/427 ━━━━━━━━━━━━━━━━━━━━ 0s 989ms/step - accuracy: 0.7345 - loss: 0.9040
Epoch 4: val_loss improved from 1.00777 to 

In [ ]:

# Loss
fig_loss = go.Figure()
fig_loss.add_trace(go.Scatter(y=history.history['loss'], name='Train Loss', mode='lines+markers'))
fig_loss.add_trace(go.Scatter(y=history.history['val_loss'], name='Val Loss', mode='lines+markers'))
fig_loss.update_layout(title='Loss over Epochs', xaxis_title='Epoch', yaxis_title='Loss', template='plotly_white')
fig_loss.show()

# Accuracy
fig_acc = go.Figure()
fig_acc.add_trace(go.Scatter(y=history.history['accuracy'], name='Train Accuracy', mode='lines+markers'))
fig_acc.add_trace(go.Scatter(y=history.history['val_accuracy'], name='Val Accuracy', mode='lines+markers'))
fig_acc.update_layout(title='Accuracy over Epochs', xaxis_title='Epoch', yaxis_title='Accuracy', template='plotly_white')
fig_acc.show()


In [ ]:
# Obtener predicciones
y_probs = model.predict(test_gen)  # probability
y_pred = np.argmax(y_probs, axis=1)  # collapse into a prediction
y_true = test_gen.classes  # real classes 

# get class names
class_names = list(test_gen.class_indices.keys())

# print Classification Report
print(classification_report(y_true, y_pred, target_names=class_names))


In [ ]:
y_probs = model.predict(test_gen)  # probability
y_pred = np.argmax(y_probs, axis=1)  # collapse into a prediction
y_true = test_gen.classes  # real classes 

# get class names
class_names = list(test_gen.class_indices.keys())

# print Classification Report
print(classification_report(y_true, y_pred, target_names=class_names))


cm = confusion_matrix(y_true, y_pred)

# Plot heatmap
fig_cm = ff.create_annotated_heatmap(
    z=cm,
    x=class_names,
    y=class_names,
    colorscale='Viridis',
    showscale=True
)

fig_cm.update_layout(
    title='Matriz de Confusión',
    xaxis_title='Predicción',
    yaxis_title='Real'
)
fig_cm.show()
